<a href="https://colab.research.google.com/github/5dongdong/PY_AI/blob/master/model/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#pip install catboost
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data/medical_noshow.csv')  

In [5]:
df

,Age,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,No-show,Handicap_0,Handicap_1,Handicap_2,Handicap_3,Handicap_4,AwaitingTime,Num_App_Missed,Gender_F,Gender_M
0,62,0,1,0,0,0,0,1,0,0,0,0,0.776481,0,1,0
1,56,0,0,0,0,0,0,1,0,0,0,0,0.672535,0,0,1
2,62,0,0,0,0,0,0,1,0,0,0,0,0.679907,0,1,0
3,8,0,0,0,0,0,0,1,0,0,0,0,0.728831,0,1,0
4,56,0,1,1,0,0,0,1,0,0,0,0,0.671794,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110514,56,0,0,0,0,1,0,1,0,0,0,0,34.614178,1,1,0
110515,51,0,0,0,0,1,0,1,0,0,0,0,34.689201,1,1,0
110516,21,0,0,0,0,1,0,1,0,0,0,0,40.330648,0,1,0
110517,38,0,0,0,0,1,0,1,0,0,0,0,40.368484,0,1,0


In [6]:
x = df.loc[:, df.columns != 'No-show']
y = df[['No-show']]   

x_train, x_test, y_train,y_test = train_test_split(
    x, y, train_size=0.8, shuffle=True, random_state=42
)
     

In [7]:
# 변수 설정
n_splits = 11   
random_state = 42
scaler = MinMaxScaler()

In [8]:
kfold = KFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler = MinMaxScaler()
scaler.fit(x_train)                 
x_train = scaler.transform(x_train)   # train 은 fit, transform 모두 해줘야 함
x_test = scaler.transform(x_test) 

In [ ]:
### GridSearchCV

'''
#parameters_01
param = {
    'learning_rate': [0.1, 0.5, 1], # controls the learning rate
    'depth': [3, 4, 5], # controls the maximum depth of the tree
    'l2_leaf_reg': [2, 3, 4], # controls the L2 regularization term on weights
    'colsample_bylevel': [0.1, 0.2, 0.3], # specifies the fraction of columns to be randomly sampled for each level
    'n_estimators': [100, 200], # specifies the number of trees to be built
    'subsample': [0.1, 0.2, 0.3], # specifies the fraction of observations to be randomly sampled for each tree
    'border_count': [32, 64, 128],# specifies the number of splits for numerical features
    'bootstrap_type': ['Bernoulli', 'MVS']
} 



#parameters_02
param = {
    'learning_rate': [0.1, 0.01, 0.001],
    'depth': [3, 5, 6, 9],
    'l2_leaf_reg': [1, 3, 5, 7]
} 

cat = CatBoostClassifier()
model = GridSearchCV(cat, param,  cv = kfold, 
                   refit = True, verbose = 1, n_jobs = -1  )

#3. 훈련
import time
start_time = time.time()
model.fit(x_train, y_train)
end_time = time.time() - start_time

print('최적의 파라미터 : ', model.best_params_ )
print('최적의 매개변수 : ', model.best_estimator_)
print('best_score : ', model.best_score_)
print('model_score : ', model.score(x_test, y_test))
print('걸린 시간 : ', end_time, '초')

'''


#model = CatBoostClassifier( depth = 7, l2_leaf_reg = 7, learning_rate =  0.05, random_state=72) 
# model_score :  0.9208288092652913



In [16]:
# 2. 모델
model = CatBoostClassifier( depth = 7, l2_leaf_reg = 7, learning_rate =  0.05, random_state=72) 

# 3. 훈련
model.fit(x_train, y_train)

#4. 평가, 예측
result = model.score(x_test,y_test)

score = cross_val_score( model, 
                        x_train, y_train,
                        cv = kfold )  #cv = cross validation_

y_predict = cross_val_predict(model,
                              x_test, y_test,
                              cv = kfold)

acc = accuracy_score(y_test, y_predict)

print('결과 acc : ', result)
print('cv pred acc : ', acc )

# 결과 acc :  0.9208288092652913

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2:	learn: 0.3903747	total: 9.9ms	remaining: 3.29s
3:	learn: 0.3403788	total: 15.1ms	remaining: 3.76s
4:	learn: 0.3049268	total: 20.4ms	remaining: 4.06s
5:	learn: 0.2774938	total: 25.6ms	remaining: 4.24s
6:	learn: 0.2558712	total: 30.9ms	remaining: 4.38s
7:	learn: 0.2393156	total: 36.1ms	remaining: 4.47s
8:	learn: 0.2268635	total: 41.4ms	remaining: 4.56s
9:	learn: 0.2172930	total: 46.5ms	remaining: 4.6s
10:	learn: 0.2093754	total: 51.8ms	remaining: 4.65s
11:	learn: 0.2028155	total: 56.7ms	remaining: 4.67s
12:	learn: 0.1972464	total: 61.8ms	remaining: 4.69s
13:	learn: 0.1931152	total: 66.8ms	remaining: 4.7s
14:	learn: 0.1895396	total: 79.1ms	remaining: 5.2s
15:	learn: 0.1867859	total: 84.5ms	remaining: 5.2s
16:	learn: 0.1843258	total: 89.8ms	remaining: 5.19s
17:	learn: 0.1822490	total: 95ms	remaining: 5.18s
18:	learn: 0.1804053	total: 100ms	remaining: 5.17s
19:	learn: 0.1787914	total: 106ms	remaining: 5.17s
20:	learn: 0.1773355	total: 111ms	remaining: 

In [14]:
def objectiveCAT(trial: Trial, x_train, y_train, x_test):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 500, 4000),
        'depth' : trial.suggest_int('depth', 8, 16),
        'fold_permutation_block' : trial.suggest_int('fold_permutation_block', 1, 256),
        'learning_rate' : trial.suggest_float('learning_rate', 0, 1),
        'od_pval' : trial.suggest_float('od_pval', 0, 1),
        'l2_leaf_reg' : trial.suggest_float('l2_leaf_reg', 0, 4),
        'random_state' :trial.suggest_int('random_state', 1, 2000)
    }
    # 학습 모델 생성
    model = CatBoostClassifier(**param)
    CAT_model = model.fit(x_train, y_train, verbose=True) # 학습 진행
    # 모델 성능 확인
    score = accuracy_score(CAT_model.predict(x_test), y_test)
    return score

In [ ]:
# MAE가 최소가 되는 방향으로 학습을 진행
# TPESampler : Sampler using TPE (Tree-structured Parzen Estimator) algorithm.
study = optuna.create_study(direction='maximize', sampler=TPESampler())

# n_trials 지정해주지 않으면, 무한 반복
study.optimize(lambda trial : objectiveCAT(trial, x, y, x_test), n_trials = 5)
print('Best trial : score {}, /nparams {}'.format(study.best_trial.value, 
                                                  study.best_trial.params))

# 하이퍼파라미터별 중요도를 확인할 수 있는 그래프
print(optuna.visualization.plot_param_importances(study))
# 하이퍼파라미터 최적화 과정을 확인
optuna.visualization.plot_optimization_history(study)
plt.show()